In [36]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

basic_file = "data/basic_stats.csv"
opp_basic_file = "data/opp_basic_stats.csv"
advanced_file = "data/advanced_stats.csv"
opp_advanced_file = "data/opp_advanced_stats.csv"

In [80]:
soup = BeautifulSoup(requests.get("https://www.sports-reference.com/cbb/seasons/2021-school-stats.html").content)
names = soup.find_all("td", attrs={"data-stat":"school_name"})
tourney_teams = [n.text.replace("NCAA", "").strip() for n in names if "NCAA" in n.text]
tourney_teams

['Abilene Christian',
 'Alabama',
 'Appalachian State',
 'Arkansas',
 'Baylor',
 'Brigham Young',
 'UC-Santa Barbara',
 'Clemson',
 'Cleveland State',
 'Colgate',
 'Colorado',
 'Connecticut',
 'Creighton',
 'Drake',
 'Drexel',
 'Eastern Washington',
 'Florida State',
 'Florida',
 'Georgetown',
 'Georgia Tech',
 'Gonzaga',
 'Grand Canyon',
 'Hartford',
 'Houston',
 'Illinois',
 'Iona',
 'Iowa',
 'Kansas',
 'Liberty',
 'Louisiana State',
 'Loyola (IL)',
 'Maryland',
 'Michigan State',
 'Michigan',
 'Missouri',
 'Morehead State',
 "Mount St. Mary's",
 'Norfolk State',
 'North Carolina-Greensboro',
 'North Carolina',
 'North Texas',
 'Ohio State',
 'Ohio',
 'Oklahoma State',
 'Oklahoma',
 'Oral Roberts',
 'Oregon State',
 'Oregon',
 'Purdue',
 'Rutgers',
 'San Diego State',
 'Southern California',
 'St. Bonaventure',
 'Syracuse',
 'Tennessee',
 'Texas Southern',
 'Texas Tech',
 'Texas',
 'UCLA',
 'Utah State',
 'Villanova',
 'Virginia Commonwealth',
 'Virginia Tech',
 'Virginia',
 'West Vi

In [71]:
drop_cols = ["Unnamed: 8", "Unnamed: 11",  "Unnamed: 14",  "Unnamed: 17", "Unnamed: 20", "W.2", "L.2", "W.3", "L.3"]
rename_dict = {'W.1':"conference W", 'L.1':"conference L", "Tm.":"Team Points", "Opp.":"Opponent Points"}

def process_df(file):
    df = pd.read_csv(file)
    df = df.drop(columns=drop_cols, axis=1).rename(columns=rename_dict)
    
    df["School"] = df["School"].apply(lambda n : n.replace("NCAA", '').strip())
    
    df = df[df["School"].isin(tourney_teams)]
    
    return df

In [72]:
basic_df = process_df(basic_file)
opp_basic_df = process_df(opp_basic_file)
advanced_df = process_df(advanced_file)
opp_advanced_df = process_df(opp_advanced_file)


In [75]:
adv_stats = list(advanced_df.columns.difference(basic_df.columns))
adv_stats.append("School")

stats_df = pd.merge(basic_df, advanced_df[adv_stats], on="School")
opp_stats_df = pd.merge(opp_basic_df, opp_advanced_df[adv_stats], on="School")


In [76]:
stats_df.to_excel("stats.xlsx")
opp_stats_df.to_excel("opp_stats.xlsx")

In [89]:
matchups =  [
    ["Texas Southern","Mount St. Mary's"],
    ["UCLA","Michigan State"],
    ["Michigan",""],
    ["Louisiana State", "St. Bonaventure"],
    ["Colorado","Georgetown"],
    ["Florida State","North Carolina-Greensboro"],
    ["Brigham Young",""],
    ["Texas","Abilene Christian"],
    ["Connecticut","Maryland"],
    ["Alabama","Iona"],
    ["Illinois","Drexel"],
    ["Loyola (IL)","Georgia Tech"],
    ["Tennessee","Oregon State"],
    ["Oklahoma State","Liberty"],
    ["San Diego State","Syracuse"],
    ["West Virginia","Morehead State"],
    ["Clemson","Rutgers"],
    ["Houston","Cleveland State"],
    ["Baylor","Hartford"],
    ["North Carolina","Wisconsin"],
    ["Villanova","Winthrop"],
    ["Purdue","North Texas"],
    ["Texas Tech","Utah State"],
    ["Arkansas","Colgate"],
    ["Florida","Virginia Tech"],
    ["Ohio State","Oral Roberts"],
    ["Gonzaga",""],
    ["Oklahoma","Missouri"],
    ["Creighton","UC-Santa Barbara"],
    ["Virginia","Ohio"],
    ["Southern California",""],
    ["Kansas","Eastern Washington"],
    ["Oregon","Virginia Commonwealth"],
    ["Iowa","Grand Canyon"]
]

In [90]:

for m in matchups:
    if(m[0] not in tourney_teams):
        print(m[0])
    if(m[1] not in tourney_teams):
        print(m[1])

In [91]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()

for m in matchups:
    t1 = m[0]
    t2 = m[1]
    
    if(t1 == "" or t2 ==""):
        continue
    
    t1_stats = stats_df[stats_df["School"] == t1]
    t1_opps = opp_stats_df[opp_stats_df["School"] == t1]
    t2_stats = stats_df[stats_df["School"] == t2]
    t2_opps = opp_stats_df[opp_stats_df["School"] == t2]
    
    df1 = df1.append(t1_stats)
    df1 = df1.append(t2_opps)
    df2 = df2.append(t2_stats)
    df2 = df2.append(t1_opps)
    
df1.to_excel("df1.xlsx")
df2.to_excel("df2.xlsx")
    